In [ ]:
import numpy as np
import numpy.linalg as la
import sympy as sp
from typing import List

In [ ]:
# General Functions

# value of f(x) when x = x0
def fx(f,x0:np.ndarray,x:List[sp.core.symbol.Symbol])->float:
  value = f
  for i in range(len(x)):
    value = value.subs(x[i], x0[i])
  return float(value)

# value of f(x) when x = x0, ND
def fx_multi(fs:np.ndarray,x0:np.ndarray,x:List[sp.core.symbol.Symbol])->np.ndarray:
  value = np.zeros(fs.shape)
  for i in range(len(value)):
    temp = fs[i]
    for j in range(len(x)):
      temp = temp.subs(x[j], x0[j])
    value[i] = float(temp)
  return value

# n-D x, return df
# 导数精确值, n维, x:[f的symbol,按x1,x2,x3..排]
def diff_multi(f,x0:np.ndarray, x:List[sp.core.symbol.Symbol])->np.ndarray:
  df = []
  for var in x:
    df.append(f.diff(var,1))
  for i in range(len(df)):
    for j in range(len(x)):
      df[i] = df[i].subs(x[j], x0[j])
  return np.array(df)

# Return the jacob matrix (not solved yet)
def get_jacob(fs: np.ndarray, x:List[sp.core.symbol.Symbol])->np.ndarray:
  n = m = fs.shape[0]
  jacob = []
  for i in range(n):
    jacob.append([])
    for j in range(m):
      jacob[i].append(fs[i].diff(x[j],1))
  return np.array(jacob) 

# Solve the given jacob matrix Jacob(x)
def solve_jacob(jacob:np.ndarray, x0:np.ndarray, x:List[sp.core.symbol.Symbol])->np.ndarray:
  n,m = jacob.shape
  result = np.zeros((n,m))
  for i in range(n):
    for j in range(m):
      expr = jacob[i][j]
      for k in range(len(x)):
        expr = expr.subs(x[k],x0[k])
      result[i][j] = expr
  return result


In [ ]:
# Bisection

# Find the length of the interval after given iteration
# Use this only when no function was given!
def length_after_iteration(a:float, b:float, iteration:int)->int:
  return np.abs(a-b)/(2**iteration)

# Use bisection to find the root of the given function
def Bisection_single(f, a:float, b:float, tol:float, iteration=-1)->list:
  num = 0
  fa = fx(f, np.array([a]),x)
  fb = fx(f, np.array([b]),x)
  m = (a+b)/2

  while((np.abs(b-a)>tol and tol!=-1) or (tol==-1 and iteration>num)):
    fa = fx(f, np.array([a]),x)
    fb = fx(f, np.array([b]),x)
    m = (a+b)/2
    fm = fx(f, np.array([m]),x)
    if(fm*fa>0):
      a = m
    elif(fm*fb>0):
      b = m
    else:
      return None
    num += 1
  return [a,b,"Interval length = "+str((a+b)/2), num]


In [ ]:
# Secant method
def Secant(f,x0:float, x1:float, x:List[sp.core.symbol.Symbol], tol: float, iteration=-1)->list:
  num = 0
  xk0 = np.copy(x0)
  xk1 = np.copy(x1)
  xk2 = 0
  while((np.abs(fx(f,np.array([xk2]),x))>tol and tol!=-1) or (tol==-1 and iteration>num)):
    f_xk1 = fx(f,np.array([xk1]),x)
    f_xk0 = fx(f,np.array([xk0]),x)
    slope = (f_xk1-f_xk0)/(xk1-xk0)
    xk2 = xk1 - (f_xk1/slope)
    xk0 = xk1
    xk1 = xk2
    num += 1
  return [xk2, num]


In [ ]:
# Newton's method

# ND Newton's method
def Newton_multi(fs:np.ndarray,x0:np.ndarray, x:List[sp.core.symbol.Symbol], tol: float, iteration=-1)->list:
  J = get_jacob(fs,x)
  xi = np.copy(x0)
  num = 0
  while True:
    Jxi = solve_jacob(J,xi,x)
    fxi = fx_multi(fs,xi,x).T 
    s = la.solve(Jxi, -1*fxi)
    xi = xi + s
    num += 1
    if((la.norm(s,np.inf)<=tol) or (iteration!=-1 and (num>=iteration))): break
  return [xi, num] # num = number of iterations

# 1D. Newton's method
def Newton_single(f,x0:np.ndarray, x:List[sp.core.symbol.Symbol], tol: float, iteration=-1)->list:
  num = 0
  xk = np.copy(x0)
  fxk = fx(f,xk,x)
  while((np.abs(fxk)>tol and tol!=-1) or (tol==-1 and iteration>num)):
    num += 1
    h = -1*fxk/diff_multi(f,xk,x)
    xk = xk+h
    fxk = fx(f,xk,x)
  return [xk, num] # num = number of iterations


In [ ]:
# Perform Two Steps of Newton's Method
# Non-linear_Equation (Find Root)
x = sp.symbols("x:1")
x0 = np.array([0])
f = x[0]**3 - 4*x[0] + 8
print(Newton_single(f,x0,x,tol=-1,iteration=1))
print(Newton_single(f,x0,x,tol=-1,iteration=2))

[array([2.00000000000000], dtype=object), 1]
[array([1.00000000000000], dtype=object), 2]


In [ ]:
# Newton Solve
# Non-linear_Equation
x = sp.symbols("x:2")
x0 = [-2.0,0.0]
f_1 = np.array([[1,0,-2,-1,-1,2],[0,-3,3,-3,-2,-3]])
f_2 = np.array([[x[0]**2],[x[0]],[1],[x[1]**2],[x[1]],[x[0]*x[1]]])
fs = f_1 @ f_2
fs = fs.flatten()
jacob = get_jacob(fs,x)
print(solve_jacob(jacob,x0,x))
print(Newton_multi(fs,x0,x,tol=10**-8,iteration=1))

[[-4. -5.]
 [-3.  4.]]
[array([-0.29032258, -0.96774194]), 1]


In [ ]:
# Determine the length of the interval
x = sp.symbols("x:1")
f = (x[0]-2)**2
Bisection_single(f, -12, 12, tol=-1, iteration=4)

[10.5, 12, 'Interval length = 11.25', 4]

In [ ]:
# NewtonSolve
# Non-linear_Equation
x = sp.symbols("x:2")
x0 = [-1.0,0.0]
fs = np.array([[4*x[0]**2+5*x[1]**3-6],[2*x[0]**4+3*x[1]-5]])
fs = fs.flatten()
jacob = get_jacob(fs,x)
print(solve_jacob(jacob,x0,x))
print(Newton_multi(fs,x0,x,tol=10**-8,iteration=1))

[[-8.  0.]
 [-8.  3.]]
[array([-1.25      ,  0.33333333]), 1]


In [ ]:
# Newton Solve 2
# Non-linear_Equation
x = sp.symbols("x:2")
x0 = [1,-2]
fs = np.array([[4*x[0]*x[1]],[x[0]**3+x[1]**2-5]])
fs = fs.flatten()
jacob = get_jacob(fs,x)
print(solve_jacob(jacob,x0,x))
print(Newton_multi(fs,x0,x,tol=10**-8,iteration=1))

[[-8.  4.]
 [ 3. -4.]]
[array([-0.6, -3.2]), 1]


In [ ]:
# Determine the length of the interval
x = sp.symbols("x:1")
f = (x[0]-4)**3
Bisection_single(f, -11, 8, tol=-1, iteration=4)

[3.25, 4.4375, 'Interval length = 3.84375', 4]

In [ ]:
# Newton's Method
x = sp.symbols("x:3")
fs = np.array([16*x[0]**4+16*x[1]**4+x[2]**4-16, x[0]**2+x[1]**2+x[2]**2-3, x[0]**3-x[1]])
get_jacob(fs, x)

array([[64*x0**3, 64*x1**3, 4*x2**3],
       [2*x0, 2*x1, 2*x2],
       [3*x0**2, -1, 0]], dtype=object)

In [ ]:
x = sp.symbols("x:2")
fs = np.array([x[0]**2+x[1]**2-1, x[0]**2-x[1]])
get_jacob(fs, x)

array([[2*x0, 2*x1],
       [2*x0, -1]], dtype=object)

In [ ]:
x = sp.symbols("x:1")
x0 = [0]
f = x[0]**3-2*x[0]-2
a = Newton_single(f,x0,x,-1, iteration = 2)
print(a)

[array([0], dtype=object), 2]


In [ ]:
x = sp.symbols("x:2")
x0 = [-2.0,0.0]
f_1 = np.array([[1,0,1,2,1,-3],[1,2,0,-1,0,2]])
f_2 = np.array([[x[0]**2],[x[0]],[1],[x[1]**2],[x[1]],[x[0]*x[1]]])
fs = f_1 @ f_2
fs = fs.flatten()
jacob = get_jacob(fs,x)
print(solve_jacob(jacob,x0,x))
print(Newton_multi(fs,x0,x,tol=10**-8,iteration=1))

[[-4.  7.]
 [-2. -4.]]
[array([-1.33333333, -0.33333333]), 1]


In [ ]:
x = sp.symbols("x:2")
fs = np.array([x[0]+6, 2*x[0]**3+x[1]-5])
x0 = np.array([-1,0])

jacob = get_jacob(fs,x)
Newton_multi(fs,x0,x,tol=10**-12,iteration=1)

[array([-6., 37.]), 1]